# TODO

* Important
    + A weight function that receives a list of graphs
    + optimize the code to consume less memory
    + Double-check if the normalization should be done for each benchmark or using all dataset
      I believe that only the weights should consider all benchmarks (excluding test).
    + Double-check if we are doing the incremental DR correctly, because the number of DRVs keeps unchanged across the iterations (check the complete routeDesign result).
    + Improve NN achitecture (parameters)
    + Include more DRVs
    + Make sure to remove completely overlapped samples.
    + Add padding in check_node_neighbors
    + Grid drawn

* Readings
    + Fit Generator hands-on: https://www.pyimagesearch.com/2018/12/24/how-to-use-keras-fit-and-fit_generator-a-hands-on-tutorial/
    + Good reading about Standardization: https://sebastianraschka.com/Articles/2014_about_feature_scaling.html
    + Standardization vs normalization: https://towardsdatascience.com/normalization-vs-standardization-quantitative-analysis-a91e8a79cebf
    + Scaling: https://scikit-learn.org/stable/modules/preprocessing.html
    + https://machinelearningmastery.com/how-to-configure-the-number-of-layers-and-nodes-in-a-neural-network/
    + https://machinelearningmastery.com/evaluate-skill-deep-learning-models/
    + https://www.microsoft.com/en-us/research/blog/three-mysteries-in-deep-learning-ensemble-knowledge-distillation-and-self-distillation/
    + https://towardsdatascience.com/how-to-handle-large-datasets-in-python-with-pandas-and-dask-34f43a897d55
    + Email: How to run GPU job on HPC cluster

# Imports

In [ ]:
import json
import math
import matplotlib
import networkx as nx
import numpy as np
import pandas as pd
import seaborn
import sklearn
import sklearn.model_selection
import sklearn.preprocessing
import tensorflow as tf
import random
import itertools

# Hyperparameters

In [ ]:
# Fix random seed.
tf.random.set_seed(1234)
np.random.seed(1234) # Scikit Learn does not have its own global random state but uses the numpy random state instead.

placementFeatures = ["NumCells", "NumCellPins", "NumMacros", "NumMacroPins", "NumPassingNets",
                     "AreaTile", "AreaCells", "AreaMacros", "AreaMacroPins",
                     "AreaL1Blkg", "AreaL2Blkg", "AreaL1Pin", "AreaL2Pin"]
GRFeatures = ["NumVerticalOverflow", "NumVerticalRemain", "NumVerticalTracks",
              "NumHorizontalOverflow", "NumHorizontalRemain", "NumHorizontalTracks"]
features = placementFeatures
features.extend(GRFeatures)

label_name = "HasDetailedRoutingViolation"

# Benchmarks
# ispd18 = ["ispd18_test"+str(x) for x in range(1, 11)]
# ispd18.extend(["ispd18_test5_metal5", "ispd18_test8_metal5"])

ispd19 = ["ispd19_test"+str(x) for x in range(1, 11)]
ispd19.extend(["ispd19_test7_metal5", "ispd19_test8_metal5", "ispd19_test9_metal5"])

circuits = ispd19
test_circuit = "ispd19_test10"
if test_circuit in circuits:
    circuits.remove(test_circuit)

# Paths
graph_path = "/home/sheiny/workspace/results/gcells/graphs/"
csv_path = "/home/sheiny/workspace/results/gcells/csvs/"

#Window Width and Height will be equal to 2 * window_radius + 1
#WARNING: The window_radius must be greater than zero
#For instance if window_radius = 7 the window size will be 15x15
window_radius = 7
window_size = 2*window_radius+1
# batch_size = 2048
# epochs = 300
# learning_rate = 0.0045 # decayed every two epochs using an exponential rate of 0.94;

batch_size = 1024 # is important to ensure that each batch has a decent chance of containing a few positive samples
epochs = 10
learning_rate = 0.005
beta = 0.001 #regularization
drop_out = 0.05

METRICS = [tf.keras.metrics.TruePositives(name='tp'),
           tf.keras.metrics.FalsePositives(name='fp'),
           tf.keras.metrics.TrueNegatives(name='tn'),
           tf.keras.metrics.FalseNegatives(name='fn'),
           tf.keras.metrics.BinaryAccuracy(name='accuracy'),
           tf.keras.metrics.Precision(name='precision'),
           tf.keras.metrics.Recall(name='recall'),
           tf.keras.metrics.AUC(name='auc')]


matplotlib.rcParams['figure.figsize'] = (12, 10)
colors = matplotlib.pyplot.rcParams['axes.prop_cycle'].by_key()['color']

# Data preprocessing and graph functions using NetworkX library

ATTENTION: If you want to deploy a model, it's critical that you preserve the preprocessing calculations.
The easiest way to implement them as layers, and attach them to your model before export.

In [ ]:
# The features will be rescaled so that they’ll have the properties of a standard normal distribution.
# mean (μ) = 0
# standard deviation (σ) = 1
def standardize(train_array, val_array, test_array=None):
    scaler = sklearn.preprocessing.StandardScaler()
    train_array = scaler.fit_transform(train_array)
    val_array = scaler.transform(val_array)
    if test_array is not None:
        test_array = scaler.transform(test_array)
        return train_array, val_array, test_array
    return train_array, val_array

def standardize_graph(graph):
    all_features = {feature:[] for feature in features}
    for node_id in graph.nodes:
        node = graph.nodes[node_id]
        for feature in features:
            all_features[feature].append(node[feature])
    all_features_np = {feature:np.array(all_features[feature]) for feature in features}
    means = {feature:all_features_np[feature].mean() for feature in features}
    stddevs = {feature:all_features_np[feature].std() for feature in features}
    del all_features_np
    for node_id in graph.nodes:
        node = graph.nodes[node_id]
        for feature in features:
            node[feature] = (node[feature]-means[feature])/stddevs[feature] if stddevs[feature] != 0 else 0

# Claculate weight for classes
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
def calculate_class_weights(df, label_name):
    neg, pos = np.bincount(df[label_name])
    total = neg + pos
    print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(total, pos, 100 * pos / total))
    weight_for_0 = (1 / neg)*(total)/2.0 
    weight_for_1 = (1 / pos)*(total)/2.0
    class_weight = {0: weight_for_0, 1: weight_for_1}
    print('Weight for class 0: {:.2f}'.format(weight_for_0))
    print('Weight for class 1: {:.2f}'.format(weight_for_1))
    return class_weight, neg, pos

def calculate_class_weights_from_graphs(graphs):
    neg = 0
    pos = 0
    for graph in graphs:
        for node_id in graph.nodes:
            node = graph.nodes[node_id]
            if node[label_name]:
                pos += 1
            else:
                neg += 1
    total = neg + pos
    print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(total, pos, 100 * pos / total))
    weight_for_0 = (1 / neg)*(total)/2.0 
    weight_for_1 = (1 / pos)*(total)/2.0
    class_weight = {0: weight_for_0, 1: weight_for_1}
    print('Weight for class 0: {:.2f}'.format(weight_for_0))
    print('Weight for class 1: {:.2f}'.format(weight_for_1))
    return class_weight, neg, pos

def grid_graph_build(json_file_path):
    graph = nx.Graph()
    nodes_jason = open(json_file_path)
    data = json.load(nodes_jason)
    nodes_jason.close()
    for j_node in data:
        node_id = j_node["id"]
        graph.add_node(node_id)
        node = graph.nodes[node_id]
        for feature in features:
            node[feature] = j_node[feature]
            node[label_name] = j_node[label_name]
    for j_node in data:
        node_id = j_node["id"]
        node = graph.nodes[node_id]
        for neighbor in ["UpNode", "DownNode", "LeftNode", "RightNode"]:
            if j_node[neighbor] != -1:
                graph.add_edge(node_id, j_node[neighbor])
            node[neighbor] = j_node[neighbor] if j_node[neighbor] != -1 else -1
    return graph

# simple sanity check function for a grid graph structure
def check_json_grid_graph(json_file_path):
    graph = grid_graph_build(json_file_path)
    nodes_jason = open(json_file_path)
    data = json.load(nodes_jason)
    nodes_jason.close()
    corners = 0
    for j_node in data:
        node_id = j_node["id"]
        node = graph.nodes[node_id]
        if graph.degree[node_id] == 0:
            print("WARNING! disconnected node found!")
        elif graph.degree[node_id] == 2:
            corners += 1
        elif graph.degree[node_id] != 3 and graph.degree[node_id] != 4:
            print("WARNING! strange node degree found: ", graph.degree[node_id])
        for feature in features:
            if node[feature] < 0 and feature not in ["NumVerticalOverflow", "NumVerticalRemain", "NumVerticalTracks",
                                                     "NumHorizontalOverflow", "NumHorizontalRemain", "NumHorizontalTracks"]:
                print('WARNING! negative feature detected!',feature)
    if corners != 4:
        print("WARNING! corners found: ", corners, " it should be 4!")

def get_valid_id_nodes(graph):
    valid_node_ids = []
    for node_id in graph.nodes:
        node = graph.nodes[node_id]
        if check_node_neighbors(graph, node, window_radius):
            valid_node_ids.append(node_id)
    return valid_node_ids

def graph_to_df(graph):
    return pd.DataFrame([graph.nodes[x] for x in graph.nodes])

def check_node_neighbors(graph, node, window_radius):
    for direction in ["UpNode", "DownNode", "LeftNode", "RightNode"]:
        current_node = node
        for x in range(window_radius):
            if current_node[direction] == -1:
                return False
            current_node = graph.nodes[current_node[direction]]
    return True

def get_window_elements(graph, node, window_radius):
    max_neighbor_hops = window_size-1
    window_elements = []
    current_node = node
    for y in range(window_radius):
        current_node = graph.nodes[current_node["UpNode"]]
    for x in range(window_radius):
        current_node = graph.nodes[current_node["LeftNode"]]
    leftmost_node = current_node
    for y in range(window_size):
        row = []
        current_node = leftmost_node
        for x in range(window_size):
            row.append([current_node[feature] for feature in features])
            if x != max_neighbor_hops:
                current_node = graph.nodes[current_node["RightNode"]]
        window_elements.append(row)
        if y != max_neighbor_hops:
                leftmost_node = graph.nodes[leftmost_node["DownNode"]]
    return window_elements

def graph_to_np_array(graph, window_radius):
    train_samples = []
    train_labels = []
    if window_radius < 0:
        print('ERROR: the window_radius must be greater than zero.')
    valid_node_ids = get_valid_id_nodes(graph)
    for node_id in valid_node_ids:
        node = graph.nodes[node_id]
        train_samples.append(get_window_elements(graph, node, window_radius))
        train_labels.append(node[label_name])
    return np.array(train_samples), np.array(train_labels)

def write_csv_from_graph(graph, output_path):
    with open(output_path, 'w') as file:
        header = ','.join([feature for feature in features])
        file.write(header+","+label_name+"\n")
        valid_node_ids = get_valid_id_nodes(graph)
        for node_id in valid_node_ids:
            node = graph.nodes[node_id]
            np_image = np.array(get_window_elements(graph, node, window_radius))
            np_image = np_image.reshape(-1)
            str_image = ','.join([str(n) for n in np_image])
            label = node[label_name]
            line = str_image+","+str(int(label))+"\n"
            file.write(line)
        file.close()

def image_generator_from_graph(mapping, graphs, batch_size, mode="train"):
    while True:
        images = []
        labels = []
        random_sequence = random.sample(range(len(mapping)), len(mapping))
        last_id = random_sequence[len(random_sequence)-1]
        for random_id in random_sequence:
            node_id = mapping[random_id][0]
            graph_id = mapping[random_id][1]
            graph = graphs[graph_id]
            node = graph.nodes[node_id]
            images.append(get_window_elements(graph, node, window_radius))
            labels.append(int(node[label_name]))
            if len(images) == batch_size:
                yield(np.array(images), np.array(labels))
                images = []
                labels = []
        if mode == "test":
            break

def image_generator_from_csv(inputPath, mode="train", aug=None):
    # open the CSV file for reading
    f = open(inputPath, "r")
    line = f.readline() #Skip Header
    # loop indefinitely
    while True:
        # initialize our batches of images and labels
        images = []
        labels = []
        # keep looping until we reach our batch size
        while len(images) < batch_size:
            # attempt to read the next line of the CSV file
            line = f.readline()
            # check to see if the line is empty, indicating we have
            # reached the end of the file
            if line == "":
                # reset the file pointer to the beginning of the file
                # and re-read the line
                f.seek(0)
                line = f.readline() #Skip Header
                line = f.readline()
                # if we are evaluating we should now break from our
                # loop to ensure we don't continue to fill up the
                # batch from samples at the beginning of the file
                if mode == "test":
                    break
            # extract the label and construct the image
            np_image = np.fromstring(line, sep=',')
            label = int(np_image[len(np_image)-1])
            np_image = np.arange(len(np_image)-1)
            np_image = np_image.reshape(window_size, window_size, len(features))
            # update our corresponding batches lists
            images.append(np_image)
            labels.append(label)
        # yield the batch to the calling function
        yield (np.array(images), np.array(labels))

# Load Training Data from JSON Grid Graphs

In [ ]:
graphs = []
for circuit in circuits:
    graph = grid_graph_build(graph_path+circuit+"_0.JSON")
    standardize_graph(graph)
    graphs.append(graph)

node_mapping = {}
idx = 0
for graph_index, graph in enumerate(graphs):
    valid_nodes = get_valid_id_nodes(graph)
    for node_id in valid_nodes:
        node_mapping[idx] = (node_id, graph_index)
        idx += 1

# Claculate weight for classes
class_weight, neg, pos = calculate_class_weights_from_graphs(graphs)

# Load Training Data From ICCAD19

In [ ]:
file_type_JSON = False

test_df = pd.DataFrame() 
dataframes = []
if file_type_JSON:
    dataframes = [graph_to_df(grid_graph_build(graph_path+circuit+'.JSON')) for circuit in circuits]
else:
    dataframes = [pd.read_csv(csv_path+circuit+'_0.csv') for circuit in circuits]
#     dataframes = [pd.read_csv(csv_path+circuit+'_1.csv') for circuit in circuits]
#     dataframes = [pd.read_csv(csv_path+circuit+'_2.csv') for circuit in circuits]
#     dataframes = [pd.read_csv(csv_path+circuit+'_3.csv') for circuit in circuits]
#     dataframes = [pd.read_csv(csv_path+circuit+'_4.csv') for circuit in circuits]
    test_df = pd.read_csv(csv_path+test_circuit+'_0.csv')

df = pd.concat(dataframes, ignore_index=True)

# Remove graph attributes
if file_type_JSON:
    df = df.drop(columns=["UpNode", "DownNode", "LeftNode", "RightNode"])
    test_df = test_df.drop(columns=["UpNode", "DownNode", "LeftNode", "RightNode"])


# df = df.drop(columns=["#VerticalOverflow", "#VerticalRemain", "#VerticalTracks", "#HorizontalOverflow", "#HorizontalRemain", "#HorizontalTracks"])
# test_df = test_df.drop(columns=["#VerticalOverflow", "#VerticalRemain", "#VerticalTracks", "#HorizontalOverflow", "#HorizontalRemain", "#HorizontalTracks"])

train_df, val_df = sklearn.model_selection.train_test_split(df, test_size=0.2)

# Form np arrays of labels and features.
train_labels = np.array(train_df.pop(label_name))
val_labels = np.array(val_df.pop(label_name))
test_labels = np.array(test_df.pop(label_name))

train_array = np.array(train_df)
val_array = np.array(val_df)
test_array = np.array(test_df)

# Scale
train_array, val_array, test_array = standardize(train_array, val_array, test_array)

# Claculate weight for classes
class_weight, neg, pos = calculate_class_weights(df, label_name)

# Load EhPredictor's dataset

In [ ]:
df = pd.read_csv("data/ISDP14/EhPredictorISPD14.csv")

# drop l53 because is always zero
df.pop('l53')
df.pop('normal')

# Instead of having the number of shorts, use them as a boolean
df.loc[df['short'] > 0, 'short'] = 1

# Convert to log-space. l9 l43 l45 l52 l51
log_cols = ['l9', 'l43', 'l45', 'l52', 'l51']
eps=0.001 # 0 => 0.1¢
for col in log_cols:
    df[col] = np.log(df[col] + eps)

# CSV organization:
# des_perf_1_dataset=all_dataset[0:5476,:]
# des_perf_a_dataset=all_dataset[5476:16928,:]
# des_perf_b_dataset=all_dataset[16928:26928,:]
# fft_1_dataset=all_dataset[26928:28864,:]
# fft_2_dataset=all_dataset[28864:32113,:]
# fft_a_dataset=all_dataset[32113:38604,:]
# fft_b_dataset=all_dataset[38604:44375,:]
# matrix_mult_1_dataset=all_dataset[44375:52656,:]
# matrix_mult_a_dataset=all_dataset[52656:69168,:]
# matrix_mult_b_dataset=all_dataset[69168:90601,:]
# pci_bridge32_a_dataset=all_dataset[90601:94170,:]
# pci_bridge32_b_dataset=all_dataset[94170:103961,:]
# superblue11_a_dataset=all_dataset[103961:175113,:]
# superblue12_dataset=all_dataset[175113:241123,:]

# Test circuits: mgc fft_2
test_df = df.iloc[28864:32113]
df2 = df[0:28864]
df3 = df[32113:]
df = pd.concat([df2, df3])

# Use a utility from sklearn to split and shuffle our dataset.
train_df, val_df = sklearn.model_selection.train_test_split(df, test_size=0.2)

# Form np arrays of labels and features.
train_labels = np.array(train_df.pop('short'))
val_labels = np.array(val_df.pop('short'))
test_labels = np.array(test_df.pop('short'))

train_array = np.array(train_df)
val_array = np.array(val_df)
test_array = np.array(test_df)

# Scaling
train_array, val_array, test_array = standardize(train_array, val_array, test_array)

# Claculate weight for classes
class_weight, neg, pos = calculate_class_weights(df, 'short')

# Convolutional Neural Network

In [ ]:
def make_model(metrics = METRICS, output_bias=None, lr=learning_rate):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
    model = tf.keras.Sequential([
                                 tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(window_size, window_size, len(features)), kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
                                 tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
                                 tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
                                 tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
                                 tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
                                 tf.keras.layers.AveragePooling2D(3, 3),
                                 tf.keras.layers.Flatten(),
                                 tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.004)),
                                 tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.004)),
                                 tf.keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias)])

    model.compile(optimizer=tf.keras.optimizers.Adam(lr=lr),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=metrics)
    return model

# Neural Network

In [ ]:
def make_model(metrics = METRICS, output_bias=None, lr=learning_rate):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
    model = tf.keras.Sequential([tf.keras.layers.Dense(20,
                                                       activation='relu', # Relu throw away negative values
                                                       kernel_regularizer=tf.keras.regularizers.l2(beta)),
                                 tf.keras.layers.Dropout(drop_out),
                                 tf.keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias)])
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=lr),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=metrics)
    return model

# Save/Load Model

In [ ]:
# initial_bias = np.log([pos/neg])
# model = make_model(output_bias = initial_bias)
# model.load_weights('./CNN_weights')

# model.save_weights('./CNN_weights')

# Train with Image Generators

In [ ]:
trainGen = image_generator_from_graph(node_mapping, graphs, batch_size, mode="train")
num_train_images = len(node_mapping)

initial_bias = np.log([pos/neg])

model = make_model(output_bias = initial_bias)
H = model.fit(x=trainGen,
              steps_per_epoch=num_train_images // batch_size,
              class_weight=class_weight,
              batch_size=batch_size,
              epochs=epochs)

# Test with Image Generators

In [ ]:
test_circuit = "ispd19_test10"
test_graph = grid_graph_build(graph_path+test_circuit+"_0.JSON")
standardize_graph(test_graph)
graphs.append(test_graph)
test_graph_idx = len(graphs)-1

test_node_mapping = {}
valid_nodes = get_valid_id_nodes(test_graph)
idx = 0
for node_id in valid_nodes:
    test_node_mapping[idx] = (node_id, test_graph_idx)
    idx += 1

In [ ]:
testGen = image_generator_from_graph(test_node_mapping, graphs, batch_size, mode="test")
baseline_results = model.evaluate(testGen, batch_size=batch_size)
metrics = calculate_metrics(model, baseline_results)
print_metrics(metrics)

# Train without image generators

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((train_array, train_labels))
train_dataset = dataset.shuffle(len(train_array)).batch(batch_size)

initial_bias = np.log([pos/neg])

model = make_model(output_bias = initial_bias)
train_history = model.fit(train_dataset,
                          batch_size=batch_size,
                          validation_data=(val_array, val_labels),
                          class_weight=class_weight,
                          epochs=epochs)

# Save Model

In [ ]:
model.save_weights('./CNN_weights')

# Confusion Matrix

In [ ]:
def plot_cm(labels, predictions, p=0.5):
    cm = sklearn.metrics.confusion_matrix(labels, predictions > p)
    matplotlib.pyplot.figure(figsize=(5,5))
    seaborn.heatmap(cm, annot=True, fmt="d")
    matplotlib.pyplot.title('Confusion matrix @{:.2f}'.format(p))
    matplotlib.pyplot.ylabel('Actual label')
    matplotlib.pyplot.xlabel('Predicted label')
    matplotlib.pyplot.show()
    
def calculate_metrics(model, results):
    m = {}
    for name, value in zip(model.metrics_names, results):
        m[name] = value
    if m['precision'] + m['recall'] != 0:
        f_score = (2 * m['precision'] * m['recall'])/(m['precision'] + m['recall'])
        m['F-score'] = f_score
    sqrt = math.sqrt((m['tp']+m['fp'])*(m['tp']+m['fn'])*(m['tn']+m['fp'])*(m['tn']+m['fn']))
    if sqrt != 0:
        mcc = (m['tp'] * m['tn'] - m['fp'] * m['fn'])/sqrt
        m['MCC'] = mcc
    return m

def print_metrics(metrics):
    for x, y in metrics.items():
        print(x,':', round(y, 2))

# Some plot functions

In [ ]:
# plot the training loss and accuracy
N = epochs
matplotlib.pyplot.style.use("ggplot")
matplotlib.pyplot.figure()
matplotlib.pyplot.plot(np.arange(0, N), H.history["loss"], label="train_loss")
# matplotlib.pyplot.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
matplotlib.pyplot.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
# matplotlib.pyplot.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
matplotlib.pyplot.title("Training Loss and Accuracy on Dataset")
matplotlib.pyplot.xlabel("Epoch #")
matplotlib.pyplot.ylabel("Loss/Accuracy")
matplotlib.pyplot.legend(loc="lower left")
matplotlib.pyplot.show()
# plt.savefig("plot.png")

print("loss:",H.history["loss"])
print("accuracy:",H.history["accuracy"])
print("tp:",H.history["tp"])
print("fp:",H.history["fp"])
print("tn:",H.history["tn"])
print("fn:",H.history["fn"])
print("precision:",H.history["precision"])
print("recall",H.history["recall"])

# Training performace

In [ ]:
baseline_results = model.evaluate(train_array, train_labels, batch_size=batch_size, verbose=0)
metrics = calculate_metrics(model, baseline_results)
print_metrics(metrics)
train_predictions_baseline = model.predict(train_array, batch_size=batch_size)
plot_cm(train_labels, train_predictions_baseline)

# Test performace Test10

In [ ]:
baseline_results = model.evaluate(test_array, test_labels, batch_size=batch_size, verbose=0)
metrics = calculate_metrics(model, baseline_results)
print_metrics(metrics)
test_predictions_baseline = model.predict(test_array, batch_size=batch_size)
plot_cm(test_labels, test_predictions_baseline)

# Test performace on ISPD14 - mgc_fft_2

In [ ]:
baseline_results = model.evaluate(test_array, test_labels, batch_size=batch_size, verbose=0)
metrics = calculate_metrics(model, baseline_results)
print_metrics(metrics)
test_predictions_baseline = model.predict(test_array, batch_size=batch_size)
plot_cm(test_labels, test_predictions_baseline)

# Drvs

In [ ]:
import re
path = "/home/sheiny/workspace/results/routed_circuits"

circuit_violations = {}
for circuit in circuits:
    circuit_violations[circuit] = []
    for itr in range(5):
        
        file = path+circuit+"/drc_rpt/"+circuit+"_"+str(itr)+".rpt"
        with open(file, "r") as file:
            first_line = file.readline()
            for last_line in file:
                pass
        drv_number = last_line.split(' ')[5]
        file.close()
        
        file = path+circuit+"/routeReport/"+circuit+"_"+str(itr)+".report"
        with open(file, "r") as file:
            lines = file.readlines()
            for line in lines:
                if re.search('Total net length = *', line):
                    wirelength = line.split(' ')[4]
                    circuit_violations[circuit].append((drv_number, wirelength))
                    break
        file.close()

dataframe = pd.DataFrame(circuit_violations)
new_collumns = ["test"+str(x) for x in range(1, 11)]
new_collumns.extend(["test5_metal5", "test8_metal5"])
dataframe.columns = new_collumns
dataframe